In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, TimeDistributed, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

# Configuration
base_dir = r'E:\Khóa luận\Data\Phase 2'
export_dir = r'E:\Khóa luận\Exports\Processed'
os.makedirs(export_dir, exist_ok=True)

# Parameters
target_col = 'CH4RAWppm'  # Target variable to predict
window_size = 72          # Lookback window (3 days at hourly resolution)
forecast_horizon = 24     # Predict next 24 hours (1 day)
batch_size = 64
epochs = 200

# Data loading with your preprocessing
def load_and_preprocess(file_path):
    df = pd.read_csv(file_path, sep=';', encoding='utf-8', low_memory=False)
    
    # Basic preprocessing
    df = df.drop_duplicates()
    if 'Date time' in df.columns:
        df['Date time'] = pd.to_datetime(df['Date time'], errors='coerce')
    
    # Handle numerical columns
    numerical_cols = [col for col in df.columns if col not in ['Timestamp', 'Date time'] 
                     and pd.api.types.is_numeric_dtype(pd.to_numeric(df[col], errors='coerce'))]
    
    for col in numerical_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df.loc[df[col] < 0, col] = np.nan
        
        # Fill missing values
        df[col] = df[col].interpolate(method='linear', limit_direction='both')
        df[col] = df[col].ffill().bfill()
        df[col] = df[col].fillna(df[col].median())
    
    # Remove columns with >50% missing/zeros
    threshold = 0.5
    cols_to_drop = [col for col in numerical_cols 
                   if (df[col].isna().sum()/len(df) > threshold) 
                   or ((df[col] == 0).sum()/len(df) > threshold]
    
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
        numerical_cols = [col for col in numerical_cols if col not in cols_to_drop]
    
    # Outlier handling
    for col in numerical_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df.loc[(df[col] < lower) | (df[col] > upper), col] = np.nan
        df[col] = df[col].interpolate(method='linear', limit_direction='both')
        df[col] = df[col].ffill().bfill()
        df[col] = df[col].fillna(df[col].median())
    
    # Make stationary
    for col in numerical_cols:
        df[col] = df[col].diff().fillna(0)
    
    # Final cleanup
    cols_all_zero = [col for col in numerical_cols if (df[col] == 0).all()]
    if cols_all_zero:
        df = df.drop(columns=cols_all_zero)
        numerical_cols = [col for col in numerical_cols if col not in cols_all_zero]
    
    return df, numerical_cols

# Sequence creation
def create_sequences(data, target, window, horizon):
    X, y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        y.append(target[i+window:i+window+horizon])
    return np.array(X), np.array(y)

# Main training loop
results = []
input_files = {
    '250500': os.path.join(base_dir, '250500.csv'),
    '500500': os.path.join(base_dir, '500500.csv'),
    '750500': os.path.join(base_dir, '750500.csv')
}

for label, file_path in input_files.items():
    print(f"\n===== Processing {label} =====")
    
    # Load and preprocess
    df, numerical_cols = load_and_preprocess(file_path)
    
    if target_col not in numerical_cols:
        print(f"Target column {target_col} not found in {label}")
        continue
    
    # Scale data
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(df[[target_col]])
    
    # Create sequences
    X, y = create_sequences(scaled_values, scaled_values, window_size, forecast_horizon)
    
    # Train-test split
    split_idx = int(0.8 * len(X))
    X_train, y_train = X[:split_idx], y[:split_idx]
    X_test, y_test = X[split_idx:], y[split_idx:]
    
    # Build model
    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True), 
                     input_shape=(window_size, 1)),
        Dropout(0.3),
        Bidirectional(LSTM(64)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(forecast_horizon)
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='mse',
                 metrics=['mae'])
    
    # Callbacks
    callbacks = [
        EarlyStopping(patience=15, restore_best_weights=True),
        ModelCheckpoint(os.path.join(export_dir, f'best_model_{label}.h5'), 
                        save_best_only=True)
    ]
    
    # Train
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=1
    )
    
    # Evaluate
    y_pred = model.predict(X_test)
    
    # Inverse scaling
    y_test_orig = np.array([scaler.inverse_transform(seq.reshape(-1, 1)).flatten() 
                          for seq in y_test])
    y_pred_orig = np.array([scaler.inverse_transform(seq.reshape(-1, 1)).flatten() 
                          for seq in y_pred])
    
    # Calculate metrics
    r2 = r2_score(y_test_orig.flatten(), y_pred_orig.flatten())
    mae = mean_absolute_error(y_test_orig.flatten(), y_pred_orig.flatten())
    rmse = np.sqrt(mean_squared_error(y_test_orig.flatten(), y_pred_orig.flatten()))
    
    results.append({
        'Concentration': label,
        'R2': r2,
        'MAE': mae,
        'RMSE': rmse
    })
    
    # Visualization
    plt.figure(figsize=(15, 6))
    for i in range(3):  # Plot 3 samples
        plt.subplot(3, 1, i+1)
        plt.plot(y_test_orig[i], label='Actual')
        plt.plot(y_pred_orig[i], label='Predicted')
        plt.title(f'Sample {i+1} - {label}')
        plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(export_dir, f'predictions_{label}.png'))
    plt.close()
    
    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training History - {label}')
    plt.legend()
    plt.savefig(os.path.join(export_dir, f'training_history_{label}.png'))
    plt.close()

# Save results
results_df = pd.DataFrame(results)
print("\nFinal Results:")
print(results_df)
results_df.to_csv(os.path.join(export_dir, 'model_results.csv'), index=False)


===== Đang xử lý nồng độ 250500 =====
Số dòng ban đầu: 11839
Số dòng sau khi loại trùng lặp: 11839
Các cột bị loại bỏ do quá nhiều thiếu/0: ['H2S_concentration', 'CH4ppm', 'HCHOmg/m3', 'TVOCppm']
Đã xuất dữ liệu đã tiền xử lý ra: E:\Khóa luận\Exports\Processed\250500_processed.csv

===== Đang xử lý nồng độ 500500 =====
Số dòng ban đầu: 8646
Số dòng sau khi loại trùng lặp: 8646
Các cột bị loại bỏ do quá nhiều thiếu/0: ['ccsTVOC', 'H2S_concentration', 'CH4ppm', 'HCHOmg/m3', 'TVOCppm']
Các cột bị loại bỏ sau differencing: ['ccseCO2']
Đã xuất dữ liệu đã tiền xử lý ra: E:\Khóa luận\Exports\Processed\500500_processed.csv

===== Đang xử lý nồng độ 750500 =====
Số dòng ban đầu: 26315
Số dòng sau khi loại trùng lặp: 26315
Các cột bị loại bỏ do quá nhiều thiếu/0: ['ccsTVOC', 'H2S_concentration', 'CH4ppm', 'HCHOmg/m3', 'TVOCppm']
Các cột bị loại bỏ sau differencing: ['ccseCO2']
Đã xuất dữ liệu đã tiền xử lý ra: E:\Khóa luận\Exports\Processed\750500_processed.csv

Đã xử lý và lưu toàn bộ biểu đồ 